### Composite operator

If an op in TF/pytorch can be expressed as a combination of existing ops in CoreML, the model can be converted with just a few lines of extra code, as shown in this example.
In this case using the Core ML custom layer functionality is not required, which involves coding up the actual implementation of the operation in swift.

#### When shall I need to use Composite operators?
1. My TensorFlow/PyTorch model has an operator which is not supported by converter natively (i.e. 1:1 mapping not present)
2. My use-case is special and require different behavior compared to default one


#### When to use Composite operators over Custom operators?
- Use Composite operators when you can represent your operator using
  existing CoreML builder API(SSA operators)
- Override behavior of existing operator completely or partially
- Improve performance with some known information for given operator
- Faster than Custom operator path
   - No need to implement Custom implementation

In [1]:
import coremltools
import numpy as np
import tensorflow as tf
from _tf_utils import *
from coremltools.models.neural_network.printer import print_network_spec

In [2]:
from packaging import version
assert version.parse(tf.__version__).major >= 2, "Requires TF 2.0 and above for this tutorial"

In [3]:
from coremltools.converters.nnv2.frontend.tensorflow.tf_op_registry import register_tf_op
from coremltools.converters.nnv2.nnv2_program.ops import CoremlBuilder as cb

In [4]:
input_shape = np.random.randint(low=1, high=6, size=4)
@make_tf2_graph({'x': input_shape})
def build_model(x):
    return tf.keras.activations.selu(x)

model, inputs, outputs = build_model
_input = np.random.randn(*input_shape).astype(np.float32)
input_values = [_input]
input_dict = dict(zip(inputs, input_values))

In [5]:
# Register TF Selu activation function
# More options:
# 1. tf_alias: list of ops to map to this function
# 2. override: If true, overrides existing implementation with current
@register_tf_op(override=True)
def Selu(context, node):
    x = context[node.inputs[0]]
    alpha = 1.6732631921768188
    lamda = 1.0507010221481323
    out_elu = cb.elu(x=x, alpha=alpha)
    out = cb.mul(x=out_elu, y=lamda, name=node.name)
    context.add(node.name, out)

In [6]:
# New defined Selu will be picked
spec = convert_tf2(model, outputs)

/Volumes/Common/work/CoreML/coreml/coremltools/coremltools/converters/nnv2/builtin_types/builtins/type_mapping.py:73: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(nptype1, np.float) and np.issubdtype(nptype2, np.int):
/Volumes/Common/work/CoreML/coreml/coremltools/coremltools/converters/nnv2/builtin_types/builtins/type_mapping.py:73: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(nptype1, np.float) and np.issubdtype(nptype2, np.int):
/Volumes/Common/work/CoreML/coreml/coremltools/coremltools/converters/nnv2/builtin_types/builtins/type_mapping.py:75: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as 

In [7]:
print("Converted mlmodel:\n")
print_network_spec(spec, style='coding')

Converted mlmodel:

Inputs:
  x [1, 3, 2, 4]
Outputs:
  Identity []


def model(x):
	elu_0 = activation (x)
	Selu = multiply (elu_0)
	Identity = multiply (Selu)
 
	return Identity


In [8]:
compare_results(spec, model, input_dict, outputs)

Output match!
